# Time Series Analysis of HiPSCats
Here we will use the lsdb framework to load ztf objects and sources then perform quick computations obtain some lightcurve information. 

We will do the same joint analysis as before, where we will find a subset of high-proper motion stars with gaia, that have a lot of observations in ztf. Here, we will join the result to the ztf source catalog, and extract lightcurve information for our subset.

In [1]:
from lsdb import lsdb_client
client = lsdb_client(dask_on_ray=True, num_workers=4)

2023-10-30 13:53:38,219	INFO worker.py:1621 -- Started a local Ray instance.
/astro/users/sdwyatt/anaconda3/envs/lsdb_demo/lib/python3.10/site-packages/dask/config.py:694: UserWarning: Configuration key "shuffle" has been deprecated. Please use "dataframe.shuffle.algorithm" instead
  warnings.warn(


In [2]:
from hipscat.inspection.almanac import Almanac
import lsdb
path_to_cloud_catalog = "abfs:///hipscat/catalogs/almanac"
storage_options = {
    "account_name" : "linccdata",
    "account_key"    : "ezBADSIGArKcI0JNHFdRfLF5S/64ZJcdrbXKbK5GJikF+YAC0hDAhMputN59HA4RS4N3HmjNZgdc+AStBFuQ6Q=="
}

cloud_almanac = Almanac(dirs=[path_to_cloud_catalog], storage_options=storage_options)
for catalog in cloud_almanac.catalogs():
    print(catalog)

gaia
ztf_dr14
ztf_source


#### Here we gather our datasets that have already been partitioned in the hipscat manner

In [3]:
gaia = lsdb.from_almanac("gaia", cloud_almanac, columns=['ra', 'dec', 'pmra', 'pmdec'])
ztf = lsdb.from_almanac("ztf_dr14", cloud_almanac, columns=['ra', 'dec', 'ps1_objid', 'nobs_g', 'nobs_r', 'nobs_i', 'mean_mag_g', 'mean_mag_r', 'mean_mag_i'])
ztf_sources = lsdb.from_almanac("ztf_source", cloud_almanac)

#### Cull the gaia through conesearch and proper motion calculation

In [4]:
import numpy as np
result = gaia.cone_search(
    ra=30,
    dec=30,
    radius=1,
).assign(
    pm=lambda x: np.sqrt(x['pmra']**2 + x['pmdec']**2)
).query(
    "pm > 100"
)

result.compute()

,ra,dec,pmra,pmdec,pm
_hipscat_index,,,,,
599985615328509952,30.752383,29.286089,191.034866,19.640969,192.041890
600006703617933312,30.503461,29.332372,25.441270,-112.110780,114.961233
600044001113931776,30.848141,29.471117,117.681613,33.155804,122.263116
600078064499556352,30.683537,29.610079,111.640021,-50.966727,122.723679
600102159266086912,30.914979,29.892982,149.741485,63.617626,162.695159
600853615334129664,31.049614,29.984640,120.516671,-58.481928,133.956724
601440514025193472,29.169068,29.594295,-48.579961,-92.744375,104.697333
601459416176263168,29.732362,29.530035,-47.757533,-88.982186,100.988175
601519129106579456,30.268854,29.477900,92.092530,-43.335331,101.779099


#### Cross match our result with ztf

In [5]:
xmatch = ztf.crossmatch(result)
xmatch.compute()

,ra_ztf_dr14,dec_ztf_dr14,ps1_objid_ztf_dr14,nobs_g_ztf_dr14,nobs_r_ztf_dr14,nobs_i_ztf_dr14,mean_mag_g_ztf_dr14,mean_mag_r_ztf_dr14,mean_mag_i_ztf_dr14,ra_gaia,dec_gaia,pmra_gaia,pmdec_gaia,pm_gaia,_DIST
_hipscat_index,,,,,,,,,,,,,,,
49373,30.741507,29.287432,143140307414235473,227,622,43,21.078733,19.565740,18.189089,30.752383,29.286089,191.034866,19.640969,192.041890,0.009581
49374,30.742162,29.284280,143140307421131702,449,593,37,20.068068,19.732326,19.613211,30.752383,29.286089,191.034866,19.640969,192.041890,0.009096
49375,30.745519,29.291056,143140307455989812,0,7,1,NaN,21.367768,20.578575,30.752383,29.286089,191.034866,19.640969,192.041890,0.007779
49376,30.762255,29.285293,143140307622352925,41,418,38,21.576769,20.424394,19.515831,30.752383,29.286089,191.034866,19.640969,192.041890,0.008648
49377,30.762289,29.288910,143140307622677253,560,710,44,16.263656,15.823932,15.687212,30.752383,29.286089,191.034866,19.640969,192.041890,0.009088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204690,30.435387,30.853531,145020304353954812,45,113,2,21.503000,21.163628,20.985513,30.432930,30.854689,99.903048,-8.680111,100.279427,0.002406
204691,30.435401,30.853510,145020304356044509,45,98,2,21.503000,21.175143,20.985513,30.432930,30.854689,99.903048,-8.680111,100.279427,0.002427
204692,30.440770,30.853761,145020304407454983,498,725,44,17.052810,16.460283,16.300037,30.432930,30.854689,99.903048,-8.680111,100.279427,0.006794


#### Next, join with ztf sources on their ps1_objectid

In [6]:
join = xmatch.join(
    ztf_sources, left_on="ps1_objid_ztf_dr14", right_on="ps1_objid"
).compute()

(raylet) Spilled 2095 MiB, 10 objects, write throughput 139 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 4330 MiB, 19 objects, write throughput 127 MiB/s.
(raylet) Spilled 8205 MiB, 35 objects, write throughput 142 MiB/s.
(raylet) Spilled 32179 MiB, 154 objects, write throughput 443 MiB/s.
(raylet) Spilled 33963 MiB, 159 objects, write throughput 434 MiB/s.


In [7]:
join.query(
    "nobs_g_ztf_dr14 > 20 and nobs_r_ztf_dr14 > 20 and nobs_i_ztf_dr14 > 20"
)

,ra_ztf_dr14,dec_ztf_dr14,ps1_objid_ztf_dr14,nobs_g_ztf_dr14,nobs_r_ztf_dr14,nobs_i_ztf_dr14,mean_mag_g_ztf_dr14,mean_mag_r_ztf_dr14,mean_mag_i_ztf_dr14,ra_gaia,...,catflags,fieldID,mag,magerr,mjd,rcID,band,Norder,Dir,Npix
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
49373,30.741507,29.287432,143140307414235473,227,622,43,21.078733,19.56574,18.189089,30.752383,...,0,604,21.240219,0.200014,59059.42842,56,g,7,0,8526
49373,30.741507,29.287432,143140307414235473,227,622,43,21.078733,19.56574,18.189089,30.752383,...,0,604,21.013107,0.183489,59080.45175,56,g,7,0,8526
49373,30.741507,29.287432,143140307414235473,227,622,43,21.078733,19.56574,18.189089,30.752383,...,0,604,20.947905,0.178744,59082.38335,56,g,7,0,8526
49373,30.741507,29.287432,143140307414235473,227,622,43,21.078733,19.56574,18.189089,30.752383,...,0,604,20.879568,0.173772,59082.45196,56,g,7,0,8526
49373,30.741507,29.287432,143140307414235473,227,622,43,21.078733,19.56574,18.189089,30.752383,...,0,604,21.259205,0.201396,59090.43569,56,g,7,0,8526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227616,30.251740,30.844494,145010302517433949,501,726,57,16.544471,16.15249,16.051840,30.252384,...,0,652,16.547066,0.014516,58514.17352,9,g,7,0,8568
227616,30.251740,30.844494,145010302517433949,501,726,57,16.544471,16.15249,16.051840,30.252384,...,0,652,16.551884,0.014533,58511.14248,9,g,7,0,8568
227616,30.251740,30.844494,145010302517433949,501,726,57,16.544471,16.15249,16.051840,30.252384,...,0,652,16.546101,0.014512,58487.18088,9,g,7,0,8568
